## Import `tf` Library

In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Embedding, Dense, Concatenate, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
import pandas as pd

## Define Model

In [2]:
# Define function to create the neural network model
def create_nn_model(vocab_size: int, embedding_dim: int, max_length: int, num_of_dense: int) -> Model:
    """
    Creates a neural network model that processes user prompts using an embedding layer,
    concatenates it with function ratings, and passes through dense layers.

    Args:
        vocab_size (int): Size of the vocabulary for embedding.
        embedding_dim (int): Dimensionality of the embedding layer.
        max_length (int): Maximum length of input sequences.
        num_of_dense (int): Number of dense layers before concatenation.

    Returns:
        Model: A compiled TensorFlow model.
    """
    # Text input (user prompt)
    text_input = Input(shape=(max_length,), name="text_input")
    embedding = Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_length)(text_input)
    flatten = Flatten()(embedding)

    # Dense layers for text input
    x = flatten
    for _ in range(num_of_dense):
        x = Dense(64, activation='relu')(x)

    # Numeric input (func_rating)
    func_rating_input = Input(shape=(1,), name="func_rating_input")
    y = Dense(32, activation='relu')(func_rating_input)

    # Concatenate both paths
    concatenated = Concatenate()([x, y])
    output = Dense(1, activation='linear', name="output")(concatenated)

    # Define and compile the model
    model = Model(inputs=[text_input, func_rating_input], outputs=output)
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])

    return model

## Prep Data

In [3]:
# Generate a larger sample dataset with 20 rows
data = {
    "user_prompt": [
        "I have just been on an international flight",
        "How can I improve my sleep quality?",
        "What are the symptoms of vitamin D deficiency?",
        "Best exercises for lower back pain relief",
        "Healthy meal ideas for weight loss",
        "How to reduce stress and anxiety naturally?",
        "What are the benefits of intermittent fasting?",
        "Tips for maintaining healthy skin",
        "How much water should I drink daily?",
        "Best ways to boost immune system",
        "Exercises to improve posture",
        "How to manage time effectively?",
        "Signs and symptoms of dehydration",
        "What foods help lower cholesterol?",
        "How to stay productive while working remotely?",
        "What are the dangers of processed foods?",
        "How to get better sleep without medication?",
        "Ways to increase energy levels naturally",
        "How does meditation help mental health?",
        "Best strategies to quit smoking"
    ],
    "func_rating": [3, 4, 2, 5, 3, 4, 5, 3, 4, 5, 3, 4, 2, 5, 3, 4, 5, 3, 4, 5],
    "custom_rating": [4, 5, 3, 4, 4, 5, 5, 4, 4, 5, 3, 5, 3, 4, 4, 3, 5, 4, 4, 5]
}

# Convert to DataFrame and display
df = pd.DataFrame(data)

### Tokenizer to Process Text

In [4]:
# Tokenize and prepare data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df["user_prompt"])
vocab_size = len(tokenizer.word_index) + 1
max_length = max([len(x.split()) for x in df["user_prompt"]])
embedding_dim = 16

# Convert text data into sequences
X_text = tokenizer.texts_to_sequences(df["user_prompt"])
X_text = pad_sequences(X_text, maxlen=max_length, padding='post')
print(X_text.shape)

# Numeric input
X_func_rating = np.array(df["func_rating"]).reshape(-1, 1)
print(X_func_rating.shape)

# Target variable
y = np.array(df["custom_rating"]).reshape(-1, 1)
print(y.shape)

(20, 8)
(20, 1)
(20, 1)


## Instantiate Model

In [5]:
# Create the model
num_of_dense_layers = 2  # Example: 2 dense layers before concatenation
model = create_nn_model(vocab_size, embedding_dim, max_length, num_of_dense_layers)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


## Training

In [6]:
%%time

# Train the model
model.fit(
    {"text_input": X_text, "func_rating_input": X_func_rating},
    y,
    epochs=200,
    batch_size=2,
    verbose=2
)

Epoch 1/200
10/10 - 3s - 349ms/step - loss: 17.1425 - mae: 4.0802
Epoch 2/200
10/10 - 0s - 6ms/step - loss: 14.7081 - mae: 3.7749
Epoch 3/200
10/10 - 0s - 6ms/step - loss: 11.8023 - mae: 3.3677
Epoch 4/200
10/10 - 0s - 6ms/step - loss: 7.7183 - mae: 2.7156
Epoch 5/200
10/10 - 0s - 6ms/step - loss: 3.4160 - mae: 1.6723
Epoch 6/200
10/10 - 0s - 6ms/step - loss: 0.8596 - mae: 0.7359
Epoch 7/200
10/10 - 0s - 5ms/step - loss: 0.7324 - mae: 0.6978
Epoch 8/200
10/10 - 0s - 6ms/step - loss: 0.3930 - mae: 0.4576
Epoch 9/200
10/10 - 0s - 6ms/step - loss: 0.3098 - mae: 0.4465
Epoch 10/200
10/10 - 0s - 6ms/step - loss: 0.1710 - mae: 0.3052
Epoch 11/200
10/10 - 0s - 6ms/step - loss: 0.1121 - mae: 0.2496
Epoch 12/200
10/10 - 0s - 6ms/step - loss: 0.0539 - mae: 0.1782
Epoch 13/200
10/10 - 0s - 6ms/step - loss: 0.0376 - mae: 0.1546
Epoch 14/200
10/10 - 0s - 6ms/step - loss: 0.0262 - mae: 0.1167
Epoch 15/200
10/10 - 0s - 6ms/step - loss: 0.0156 - mae: 0.0872
Epoch 16/200
10/10 - 0s - 6ms/step - loss: 0

In [7]:
# Display the model summary
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ text_input (InputLayer)   │ (None, 8)              │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding (Embedding)     │ (None, 8, 16)          │          1,376 │ text_input[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ flatten (Flatten)         │ (None, 128)            │              0 │ embedding[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense (Dense)             │ (None, 64)             │          8,256 │ flatten[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ func_rating_input         │ (None, 1)              │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_1 (Dense)           │ (None, 64)             │          4,160 │ dense[0][0]            │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_2 (Dense)           │ (None, 32)             │             64 │ func_rating_input[0][… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate (Concatenate) │ (None, 96)             │              0 │ dense_1[0][0],         │
│                           │                        │                │ dense_2[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ output (Dense)            │ (None, 1)              │             97 │ concatenate[0][0]      │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 41,861 (163.52 KB)

 Trainable params: 13,953 (54.50 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 27,908 (109.02 KB)

## Create Repo on HF

In [8]:
! pip install --upgrade huggingface_hub

In [18]:
from huggingface_hub import HfApi

hf_username = "eagle0504"  # Replace with your Hugging Face username
repo_name = "my-tf-nn-model-v4"  # Change this as needed
repo_id = f"{hf_username}/{repo_name}"

api = HfApi()

# Create the repository
api.create_repo(repo_id=repo_id, exist_ok=True)  # exist_ok=True prevents errors if the repo already exists

print(f"Repository created: https://huggingface.co/{repo_id}")

Repository created: https://huggingface.co/eagle0504/my-tf-nn-model-v4


## Push Artifacts on HF

In [10]:
import pickle
import os
from huggingface_hub import HfApi
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer

# Define save directory
model_dir = "./saved_model"
os.makedirs(model_dir, exist_ok=True)

# Save the model
model_path = os.path.join(model_dir, "model.keras")
model.save(model_path)

# Save the tokenizer as a pickle file
tokenizer_path = os.path.join(model_dir, "tokenizer.pkl")
with open(tokenizer_path, "wb") as f:
    pickle.dump(tokenizer, f)

# Upload model and tokenizer to Hugging Face
repo_id = f"eagle0504/{repo_name}"  # Change as needed
api = HfApi()

api.upload_file(path_or_fileobj=model_path, path_in_repo="model.keras", repo_id=repo_id)
api.upload_file(path_or_fileobj=tokenizer_path, path_in_repo="tokenizer.pkl", repo_id=repo_id)

print(f"Model and tokenizer successfully pushed to: https://huggingface.co/{repo_id}")

model.keras:   0%|          | 0.00/212k [00:00<?, ?B/s]

tokenizer.pkl:   0%|          | 0.00/2.65k [00:00<?, ?B/s]

Model and tokenizer successfully pushed to: https://huggingface.co/eagle0504/my-tf-nn-model-v3


## Download Model Artifact from HF

In [11]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from huggingface_hub import hf_hub_download

# Set Keras backend to JAX (Optional, only needed if you want to experiment with JAX backend)
os.environ["KERAS_BACKEND"] = "jax"

# Define repo ID and model filename
# repo_id = "eagle0504/my-tf-nn-model" # we already defined it above
filename = "model.keras"  # Ensure this matches what was uploaded

# Download the model
model_path = hf_hub_download(repo_id=repo_id, filename=filename)

# Load the Keras model
new_model = keras.models.load_model(model_path)

print("Model loaded successfully!")

model.keras:   0%|          | 0.00/212k [00:00<?, ?B/s]

Model loaded successfully!


In [12]:
new_model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ text_input (InputLayer)   │ (None, 8)              │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding (Embedding)     │ (None, 8, 16)          │          1,376 │ text_input[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ flatten (Flatten)         │ (None, 128)            │              0 │ embedding[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense (Dense)             │ (None, 64)             │          8,256 │ flatten[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ func_rating_input         │ (None, 1)              │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_1 (Dense)           │ (None, 64)             │          4,160 │ dense[0][0]            │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_2 (Dense)           │ (None, 32)             │             64 │ func_rating_input[0][… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate (Concatenate) │ (None, 96)             │              0 │ dense_1[0][0],         │
│                           │                        │                │ dense_2[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ output (Dense)            │ (None, 1)              │             97 │ concatenate[0][0]      │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 41,861 (163.52 KB)

 Trainable params: 13,953 (54.50 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 27,908 (109.02 KB)

## Local Inference

In [17]:
import pickle
from huggingface_hub import hf_hub_download
from tensorflow import keras
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

# Define repo details
# repo_id = "eagle0504/my-tf-nn-model" # we already defined above

# Download model
model_path = hf_hub_download(repo_id=repo_id, filename="model.keras")
new_model = keras.models.load_model(model_path)

# Download tokenizer
tokenizer_path = hf_hub_download(repo_id=repo_id, filename="tokenizer.pkl")

# Load the tokenizer
with open(tokenizer_path, "rb") as f:
    tokenizer = pickle.load(f)

print("Model and tokenizer loaded successfully!")

# Sample test data
test_texts = [
    "How to improve focus and concentration?",
    "What are the side effects of lack of sleep?",
]

# Preprocess test data using the loaded tokenizer
max_length = model.input_shape[0][1]  # Get max_length from model input shape
X_text_test = tokenizer.texts_to_sequences(test_texts)
X_text_test = pad_sequences(X_text_test, maxlen=max_length, padding='post')

# Dummy numeric input (func_rating)
X_func_test = np.array([4, -5]).reshape(-1, 1)

# Make predictions
predictions = new_model.predict({"text_input": X_text_test, "func_rating_input": X_func_test})

# Display results
for i, text in enumerate(test_texts):
    print(f"Prompt: {text}")
    print(f"Predicted Rating: {predictions[i][0]:.2f}")
    print("-" * 50)

Model and tokenizer loaded successfully!


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 245ms/step
Prompt: How to improve focus and concentration?
Predicted Rating: 4.15
--------------------------------------------------
Prompt: What are the side effects of lack of sleep?
Predicted Rating: 2.88
--------------------------------------------------
